In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import vae_cyc as vc
import pandas as pd 
import lightning as pl
import wandb
from torch.utils.data import DataLoader 
from pytorch_lightning.loggers import WandbLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('/workspace/uniref50_small.csv')

In [4]:
vocab = vc.vocab.AminoAcidVocab()

In [5]:
train_df, val_df = train_test_split(df, test_size=0.05)

In [6]:
train_dataset = vc.Dataset(train_df, vocab, 'Sequence',100)

Tokenizing...


100%|██████████| 949999/949999 [00:55<00:00, 17177.23it/s]

Tokenization complete


In [7]:
val_dataset = vc.Dataset(val_df, vocab, 'Sequence',100)

Tokenizing...


100%|██████████| 50000/50000 [00:03<00:00, 14730.63it/s]

Tokenization complete


In [16]:
train_dl = DataLoader(train_dataset,batch_size=128,shuffle=True, collate_fn=train_dataset.collate,num_workers=95)
val_dl = DataLoader(val_dataset, batch_size=128, shuffle=False, collate_fn=val_dataset.collate, num_workers=95)

In [17]:
wandb_logger = WandbLogger(project="run_pod", log_model=True)
wandb.init()

epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁████████████████
train/global_step,▁▁▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▆▇████
train/kl_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅█▆▆▆▆▂▄▆▆▆▅
train/kl_weight,██▁█▆▁█████▁██▁███▁▃██▁████▁███▆███▃████
train/loss,█▆▁▃▃▆▇▇▇▅▃▅▆▆▅▄▅▅▇▄▁▄▄▇▆
train/lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/r_loss,█▅▆▂▃▄▄▂▅▄▅▃▃▁▁▄▃▂▂▄▃▄▂▁▇
trainer/global_step,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇██
val/kl_loss,▁
val/loss,▁
val/r_loss,▁


In [18]:
checkpoint_callback = ModelCheckpoint(dirpath='/workspace/uniref50-vae/', verbose=True, save_top_k=3,monitor='val/loss')

In [19]:
trainer = pl.Trainer(devices=1, logger=wandb_logger, precision='16-mixed', callbacks=[checkpoint_callback])

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [20]:
model = vc.VAE(256,512,4, enc_num_layers=6, vocab=vocab, c_step=50000, n_cycles_anneal=100, lr=0.001)

In [ ]:
trainer.fit(model, train_dl, val_dl)

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loops/utilities.py:72: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | encoder       | GRU              | 25.3 M | train
1 | decoder       | GRU              | 6.0 M  | train
2 | mu_fc         | Linear           | 1.6 M  | train
3 | logvar_fc     | Linear           | 1.6 M  | train
4 | embedding     | Embedding        | 928    | train
5 | latent2hidden | Linear           | 526 K  | train
6 | outputs2vocab | Linear           | 14.9 K | train
7 | crit

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:02<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]